<a href="https://colab.research.google.com/github/Karthick47v2/mock-buddy/blob/main/haar_cascade_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# image processing
import cv2

# numpy array
import numpy as np

# import model
import tensorflow
from tensorflow import keras

In [2]:
# sound recording
import pyaudio

# save wav file
import wave

In [3]:
# sound recording
import pyaudio

# save wav file
import wave

In [4]:
img_size = 96
edge_offset = 75

In [5]:
sample_rate = 44100
frames_per_buffer = 1024
no_of_channels = 1

In [6]:
model = keras.models.load_model('./model')

In [7]:
def draw_landmark(frame):
  landmarks = model.predict(frame.reshape(1,img_size,img_size,1))

  x = np.reshape(landmarks[:,:,:,np.arange(0,136,2)], (68)) * img_size
  y = np.reshape(landmarks[:,:,:,np.arange(1,136,2)], (68)) * img_size

  for x_coord, y_coord in zip(x,y):
    cv2.circle(frame, tuple((int(x_coord),int(y_coord))), radius=1, color=(255,0,0), thickness=1)

  return frame

In [8]:
def audio_func():
    #
    audio = pyaudio.PyAudio()
    stream = audio.open(format=pyaudio.paInt16, channels=no_of_channels,
                        rate=sample_rate, frames_per_buffer=frames_per_buffer, input=True)

    audio_frames = []
    i  = 0
    while i < 1000:
        print(i)
        i += 1
        audio_frames.append(stream.read(frames_per_buffer))

    stream.stop_stream()
    stream.close()
    audio.terminate()

    sound_file = wave.open("rec.wav", "wb")
    sound_file.setnchannels(no_of_channels)
    sound_file.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
    sound_file.setframerate(sample_rate)
    sound_file.writeframes(b''.join(audio_frames))
    sound_file.close()


In [9]:
def video_func():
  # download prebuilt haarcascade xml for face detection
  face_cascade = cv2.CascadeClassifier('haarcascade_xml/haarcascade_frontalface_alt2.xml')

  # webcam
  cap = cv2.VideoCapture(0)
  i = 0
  while i < 1000:
    i += 1
    ret, frame = cap.read()
    
    # haar cascade needs grayscale image so covert bgr to gray (Opencv uses bgr format)
    gray_scale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detectMultiScale returns coords of bounding box(s)
    faces_rect = face_cascade.detectMultiScale(gray_scale, scaleFactor=1.4, minNeighbors=3, minSize=(30,30))       #1.3 - scalefactor
      
    areas = [w*h for x,y,w,h in faces_rect]
    try:
        large_rect = np.argmax(areas)
        bbox = faces_rect[large_rect]

        # extending bounding box a bit in order to create input for model
        bbox[2] += bbox[0] + edge_offset
        bbox[3] += bbox[1] + edge_offset
        bbox[0] -= edge_offset
        bbox[1] -= edge_offset
        
        # cliping to make sure dims don't exceed frame range
        X = np.clip(bbox[[0,2]], 0, frame.shape[1])
        Y = np.clip(bbox[[1,3]], 0, frame.shape[0])

        # roi - region of interest 
        # offsetting bounding box because model needs full face image
        input_frame = np.array(gray_scale[Y[0] : Y[1], X[0] : X[1]])
        
        frame_h, frame_w = input_frame.shape[:2]
        scale_h, scale_w = img_size / frame_h, img_size / frame_w

        # resize to 96x96
        input_frame= cv2.resize(input_frame, (img_size, img_size)).astype(np.uint8)
        
        #inference
        frame = draw_landmark(input_frame)
        # print(model.predict(input_frame).shape)

    except:
      ##########################
      person_not_facing = True
      ##########################

    cv2.imshow('frame', frame)

    # return if q pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

  # release webcam
  cap.release()

  # close window
  cv2.destroyAllWindows()


In [58]:
# if __name__ == '__main__':

#     audio_cpu = mp.Process(target=audio_func)
#     video_cpu = mp.Process(target=video_func)
#     audio_cpu.start()
#     video_cpu.start()


In [10]:
audio_func()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27